In [ ]:
from numba import cuda
import numpy as np

In [ ]:
@cuda.jit
def zeeman_energy(grid, zeeman_K, m0, Ms, dx, dy, dz, result):
    x, y, z = cuda.grid(3)
    if zeeman_K is None:
        result[0] = 0
        return

    shape = grid.shape
    if len(shape) < 2 or len(shape) > 4:
        raise ValueError("Grid shape is not supported")

    if x < shape[0] and y < shape[1] and z < shape[2]:
        energy_sum = -m0 * Ms * (grid[x, y, z] * zeeman_K[x, y, z]) * dx * dy * dz
        cuda.atomic.add(result, 0, energy_sum)

@cuda.jit
def anisotropic_energy(grid, anisotropic_u, anisotropic_K, dx, dy, dz, result):
    x, y = cuda.grid(2)
    if anisotropic_K is None:
        result[0] = 0
        return

    shape = grid.shape
    if x < shape[0] and y < shape[1]:
        energy_dot = np.dot(grid[x, y, :], anisotropic_u[::-1])
        energy_square = energy_dot ** 2
        energy_sum = anisotropic_K * energy_square * dx * dy * dz
        cuda.atomic.add(result, 0, energy_sum)


In [ ]:

@cuda.jit
def exchange_energy(grid, exchange_A, dx, dy, dz, result):
    x, y, z = cuda.grid(3)
    shape = grid.shape

    if exchange_A is None:
        result[0] = 0
        return

    if x < shape[0] - 1 and y < shape[1] - 1 and z < shape[2] - 1:
        laplacian_M = cuda.local.array(shape=(3,), dtype=float64)

        for i in range(3):
            laplacian_M[i] = (
                (grid[x + 1, y, z, i] - 2 * grid[x, y, z, i] + grid[x - 1, y, z, i]) / (dx * dx)
                + (grid[x, y + 1, z, i] - 2 * grid[x, y, z, i] + grid[x, y - 1, z, i]) / (dy * dy)
                + (grid[x, y, z + 1, i] - 2 * grid[x, y, z, i] + grid[x, y, z - 1, i]) / (dz * dz)
            )

        energy_dot = 0
        for i in range(3):
            energy_dot += grid[x, y, z, i] * laplacian_M[i]

        energy_sum = -exchange_A * energy_dot * dx * dy * dz
        cuda.atomic.add(result, 0, energy_sum)


In [ ]:

@cuda.jit
def dmi_energy(grid, dmi_D, dx, dy, dz, type_string, result):
    x, y, z = cuda.grid(3)
    shape = grid.shape

    if dmi_D is None:
        result[0] = 0
        return

    if x < shape[0] - 1 and y < shape[1] - 1 and z < shape[2] - 1:
        # Implementing the 'Cnv_z' case as an example
        if type_string == 'Cnv_z':
            gradM_z = cuda.local.array(shape=(3,), dtype=np.float64)
            gradM_z[0] = (grid[x + 1, y, z, 2] - grid[x - 1, y, z, 2]) / (2 * dx) # Approximation for dmz/dx
            gradM_z[1] = (grid[x, y + 1, z, 2] - grid[x, y - 1, z, 2]) / (2 * dy) # Approximation for dmz/dy
            gradM_z[2] = (grid[x, y, z + 1, 2] - grid[x, y, z - 1, 2]) / (2 * dz) # Approximation for dmz/dz

            div_M = (grid[x + 1, y, z, 0] - grid[x - 1, y, z, 0]) / (2 * dx) + \
                    (grid[x, y + 1, z, 1] - grid[x, y - 1, z, 1]) / (2 * dy) + \
                    (grid[x, y, z + 1, 2] - grid[x, y, z - 1, 2]) / (2 * dz)

            m_del_mz = grid[x, y, z, 0] * gradM_z[0] + grid[x, y, z, 1] * gradM_z[1] + grid[x, y, z, 2] * gradM_z[2]
            mz_div_m = grid[x, y, z, 2] * div_M

            energy_sum = dmi_D * (m_del_mz - mz_div_m) * dx * dy * dz
            cuda.atomic.add(result, 0, energy_sum)
